# Walk-Forward with backtest_framework (1d data)

This notebook shows how to run the class-based walk-forward on daily data.

In [80]:
import pandas as pd
import numpy as np
from pathlib import Path

from backtest_framework import (
    DataBundle, MomentumIDParams, MomentumIDStrategy, LongShortVolWeighting, WalkForwardRunner,
    BacktestEngine, compute_sharpe, compute_sortino_ratio, compute_calmar_ratio, compute_composite_score, select_score
)
from binance_data_loader import BinanceDataLoader



## Load data
Adjust paths/filters as needed for your 1d parquet files.

In [81]:

# Configure loader for daily data
data_loader = BinanceDataLoader(
    data_directory="/Users/chinjieheng/Documents/data/binance_dailydata",
    timeframe="1d",
    min_records=60,
    min_volume=1e5,
    start_date="2022-09-01",
    end_date=None,
)

price = data_loader.get_price_matrix()
# Rolling volume for universe selection (20d avg)
volume_data = {t: data_loader._crypto_universe[t]['data']['volume'].reindex(price.index) for t in data_loader.get_universe()}
volume_df = pd.DataFrame(volume_data, index=price.index)
rolling_volume_df = volume_df.rolling(window=20, min_periods=10).mean()

# BTC 90d return filter
btc_90d_return = price['BTCUSDT'].pct_change(90, fill_method=None) if 'BTCUSDT' in price.columns else None


Loading Binance data from /Users/chinjieheng/Documents/data/binance_dailydata (timeframe=1d)...
Found 594 USDT trading pairs
Using a 30-bar rolling window for 30d volume checks
✓ BTCUSDT loaded successfully with 1174 records, avg volume: 14,911,837,860
Loaded 539 cryptocurrencies
Filtered 53 cryptocurrencies (insufficient data/volume)
Precomputing returns matrix (FAST numpy version)...
Building returns matrix for 539 tickers over 1174 dates...
Precomputed returns matrix shape: (1174, 539)
Date range: 2022-09-01 00:00:00 to 2025-11-17 00:00:00


## Build DataBundle
Precompute for the windows used in the param grid.

In [82]:

# Parameter grids (daily bars)
simple_windows = [12,14]
vol_windows = [15,20,25,30,35,40,45]

bundle = DataBundle(
    price_df=price,
    rolling_volume_df=rolling_volume_df,
    btc_ret=btc_90d_return,
    min_hist_days=30
)
# Precompute shared matrices
bundle.ensure_simple_returns(simple_windows)
bundle.ensure_id_matrix(simple_windows)
bundle.ensure_vol_matrix(vol_windows)


## Define parameter grid and strategy factory

In [83]:


from itertools import product

# Choices for cartesian grid
grid_choices = {
    "volume_pct": [0.2],
    "momentum_pct": [0.1],
    "long_id_threshold": [-0.8],
    "short_id_threshold": [-0.4],
    "max_positions_per_side": [10],
    "max_position_cap": [0.3],
    "tc_bps": [5],
    "use_btc_filter": [True],
}

choices = list(product(
    grid_choices["volume_pct"],
    grid_choices["momentum_pct"],
    grid_choices["long_id_threshold"],
    grid_choices["short_id_threshold"],
    grid_choices["max_positions_per_side"],
    grid_choices["max_position_cap"],
    grid_choices["tc_bps"],
    grid_choices["use_btc_filter"],
))

param_grid = []
for sw in simple_windows:
    for vw in vol_windows:
        for vc, mc, lt, st, mpps, mcap, tc, btc in choices:
            param_grid.append(
                MomentumIDParams(
                    simple_window=sw,
                    id_window=sw,
                    vol_window=vw,
                    volume_pct=vc,
                    momentum_pct=mc,
                    long_id_threshold=lt,
                    short_id_threshold=st,
                    max_positions_per_side=mpps,
                    max_position_cap=mcap,
                    min_weight=0.05,
                    tc_bps=tc,
                    use_btc_filter=btc,
                )
            )

print(f"Param grid size: {len(param_grid)}")

# Strategy factory closure
def make_strategy(params):
    return MomentumIDStrategy(params)

weighting_model = LongShortVolWeighting()



Param grid size: 14


## Configure walk-forward
All spans in bars (days for 1d data).

In [84]:

train_span = 365  # e.g., ~9 months
test_span = 90    # e.g., ~3 months
step_span = 90    # step size

runner = WalkForwardRunner(
    periods_per_year=365,
    bundle=bundle,
    strategy_factory=make_strategy,
    weighting_model=weighting_model,
    params_grid=param_grid,
    train_span=train_span,
    test_span=test_span,
    step_span=step_span,
    mode="expanding",
    score_mode="composite",
    n_jobs=10,  # set >1 to thread across params
)



## Run walk-forward

In [85]:
wf_df, oos_returns, oos_equity = runner.run()

print(wf_df[[
    'iteration', 'train_start', 'train_end', 'test_start', 'test_end',
    'is_score', 'oos_score', 'is_sharpe', 'oos_sharpe'
]])
print("Combined OOS Sharpe:", compute_sharpe(oos_returns, periods_per_year=runner.periods_per_year) if len(oos_returns) > 1 else float('nan'))

report = runner.report(
    wf_df=wf_df,
    oos_returns=oos_returns,
    oos_equity=oos_equity,
    plot=True,
    fig_dir="figures",
)

combined_equity = report.get('combined_equity', oos_equity)
iter_stats_df = report.get('iter_stats')
combined_returns_recomputed = report.get('combined_returns_recomputed')
combined_turnover = report.get('combined_turnover')


   iteration train_start  train_end test_start   test_end  is_score  \
0          1  2022-09-01 2023-09-01 2023-09-01 2023-11-29  1.845379   
1          2  2022-09-01 2023-11-30 2023-11-30 2024-02-27  0.901205   
2          3  2022-09-01 2024-02-28 2024-02-28 2024-05-27  1.480906   
3          4  2022-09-01 2024-05-28 2024-05-28 2024-08-25  1.788467   
4          5  2022-09-01 2024-08-26 2024-08-26 2024-11-23  1.485301   
5          6  2022-09-01 2024-11-24 2024-11-24 2025-02-21  1.530887   
6          7  2022-09-01 2025-02-22 2025-02-22 2025-05-22  1.843109   
7          8  2022-09-01 2025-05-23 2025-05-23 2025-08-20  1.905696   
8          9  2022-09-01 2025-08-21 2025-08-21 2025-11-17  2.226533   

   oos_score  is_sharpe  oos_sharpe  
0  -2.303054   1.050911   -1.706661  
1   3.608430   0.581889    1.913857  
2   5.975192   0.942412    2.360134  
3  -1.322050   1.126495   -1.044467  
4   2.886425   0.975645    1.809666  
5   6.535914   1.025702    3.096999  
6   7.998565   1.234910

/Users/chinjieheng/Documents/research/mom_research/backtest_framework.py:974: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_equity = btc_prices.fillna(method="ffill") / initial_btc


## Save results (optional)

In [86]:
wf_df.to_csv('walkforward_results_class.csv', index=False)
oos_returns.to_csv('walkforward_oos_returns.csv', header=False)
oos_equity.to_csv('walkforward_oos_equity.csv', header=False)
if 'iter_stats_df' in locals() and iter_stats_df is not None and not iter_stats_df.empty:
    iter_stats_df.to_csv('walkforward_iter_stats.csv', index=False)
if 'combined_turnover' in locals() and combined_turnover is not None and not combined_turnover.empty:
    combined_turnover.to_csv('walkforward_oos_turnover.csv', header=False)
